In [1]:
#Importar e instalar
import pandas as pd
import numpy as np
import pylab as plt   
import seaborn as sns
import datetime
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import OneHotEncoder
from scipy.interpolate import interp1d
from sklearn.impute import SimpleImputer
from statsmodels.tsa.arima.model import ARIMA
import warnings
#%pip install statsmodels
warnings.filterwarnings('ignore')   # para quitar esos prints

In [2]:
data = pd.read_csv('data/fusion_MG_UNICEF_1.0_all.csv', sep=',', encoding='utf-8')
#dataglob = pd.read_csv('data/fusion_GLOBAL_DATAFLOW_UNICEF_1.0_all.csv', sep=',', encoding='utf-8')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37699 entries, 0 to 37698
Data columns (total 18 columns):
 #   Column                                                                          Non-Null Count  Dtype  
---  ------                                                                          --------------  -----  
 0   DATAFLOW                                                                        37699 non-null  object 
 1   REF_AREA:Geographic area                                                        37699 non-null  object 
 2   INDICATOR:Indicator                                                             37699 non-null  object 
 3   AGE:Current age                                                                 37699 non-null  object 
 4   STAT_POP:Statistical Population                                                 37699 non-null  object 
 5   TIME_PERIOD:Time period                                                         37699 non-null  int64  
 6   OBS_VALUE:Obse

In [4]:
# Eliminar las columnas con valores nulos
df = data.dropna(axis=1)

In [5]:
# Verificar los resultados
df.head()

,DATAFLOW,REF_AREA:Geographic area,INDICATOR:Indicator,AGE:Current age,STAT_POP:Statistical Population,TIME_PERIOD:Time period,OBS_VALUE:Observation Value,UNIT_MEASURE:Unit of measure,SOURCE_LINK:Citation of or link to the data source,DATA_SOURCE:Data Source,OBS_FOOTNOTE:Observation footnote
0,UNICEF:MG(1.0): Migration,BDI: Burundi,MG_INTNL_MG_CNTRY_DEST: International migrants...,_T: Total,_T: Total,1990,333,PS: Persons,https://www.un.org/development/desa/pd/content...,United Nations Department of Economic and Soci...,235 countries/areas where migration data avail...
1,UNICEF:MG(1.0): Migration,BDI: Burundi,MG_INTNL_MG_CNTRY_DEST: International migrants...,_T: Total,_T: Total,1995,255,PS: Persons,https://www.un.org/development/desa/pd/content...,United Nations Department of Economic and Soci...,235 countries/areas where migration data avail...
2,UNICEF:MG(1.0): Migration,BDI: Burundi,MG_INTNL_MG_CNTRY_DEST: International migrants...,_T: Total,_T: Total,2000,126,PS: Persons,https://www.un.org/development/desa/pd/content...,United Nations Department of Economic and Soci...,235 countries/areas where migration data avail...
3,UNICEF:MG(1.0): Migration,BDI: Burundi,MG_INTNL_MG_CNTRY_DEST: International migrants...,_T: Total,_T: Total,2005,193,PS: Persons,https://www.un.org/development/desa/pd/content...,United Nations Department of Economic and Soci...,235 countries/areas where migration data avail...
4,UNICEF:MG(1.0): Migration,BDI: Burundi,MG_INTNL_MG_CNTRY_DEST: International migrants...,_T: Total,_T: Total,2010,247,PS: Persons,https://www.un.org/development/desa/pd/content...,United Nations Department of Economic and Soci...,235 countries/areas where migration data avail...


In [6]:
# Eliminar las filas que contienen "total" en la columna "AGE"
df = df[~df['AGE:Current age'].str.contains('_T: Total', case=False)]

In [7]:
# Eliminar la columna "DATAFLOW"
df = df.drop("DATAFLOW", axis=1)

In [8]:
# Contar la frecuencia de cada valor en la columna 'UNIT_MEASURE:Unit of measure'
unit_measure_counts = df['UNIT_MEASURE:Unit of measure'].value_counts()
print(unit_measure_counts)

PS: Persons       8286
NUMBER: Number    6351
Name: UNIT_MEASURE:Unit of measure, dtype: int64


In [9]:
# Filtrar el DataFrame original manteniendo solo las filas con 'UNIT_MEASURE:Unit of measure' igual a 'PS: Persons'
df = df[df['UNIT_MEASURE:Unit of measure'] == 'PS: Persons']

# Mostrar el DataFrame actualizado
df.head()

,REF_AREA:Geographic area,INDICATOR:Indicator,AGE:Current age,STAT_POP:Statistical Population,TIME_PERIOD:Time period,OBS_VALUE:Observation Value,UNIT_MEASURE:Unit of measure,SOURCE_LINK:Citation of or link to the data source,DATA_SOURCE:Data Source,OBS_FOOTNOTE:Observation footnote
1742,AFG: Afghanistan,MG_INTNL_MG_CNTRY_DEST: International migrants...,Y0T17: Under 18 years old,_T: Total,1990,15,PS: Persons,https://www.un.org/development/desa/pd/content...,United Nations Department of Economic and Soci...,235 countries/areas where migration data avail...
1743,AFG: Afghanistan,MG_INTNL_MG_CNTRY_DEST: International migrants...,Y0T17: Under 18 years old,_T: Total,1995,18,PS: Persons,https://www.un.org/development/desa/pd/content...,United Nations Department of Economic and Soci...,235 countries/areas where migration data avail...
1744,AFG: Afghanistan,MG_INTNL_MG_CNTRY_DEST: International migrants...,Y0T17: Under 18 years old,_T: Total,2000,19,PS: Persons,https://www.un.org/development/desa/pd/content...,United Nations Department of Economic and Soci...,235 countries/areas where migration data avail...
1745,AFG: Afghanistan,MG_INTNL_MG_CNTRY_DEST: International migrants...,Y0T17: Under 18 years old,_T: Total,2005,22,PS: Persons,https://www.un.org/development/desa/pd/content...,United Nations Department of Economic and Soci...,235 countries/areas where migration data avail...
1746,AFG: Afghanistan,MG_INTNL_MG_CNTRY_DEST: International migrants...,Y0T17: Under 18 years old,_T: Total,2010,25,PS: Persons,https://www.un.org/development/desa/pd/content...,United Nations Department of Economic and Soci...,235 countries/areas where migration data avail...


In [10]:
df = df.drop('SOURCE_LINK:Citation of or link to the data source',axis=1)

In [11]:
df = df.drop('DATA_SOURCE:Data Source', axis=1)

In [12]:
df = df.drop('OBS_FOOTNOTE:Observation footnote', axis=1)

In [13]:
df = df.drop('UNIT_MEASURE:Unit of measure', axis=1)

In [14]:
df.head()

,REF_AREA:Geographic area,INDICATOR:Indicator,AGE:Current age,STAT_POP:Statistical Population,TIME_PERIOD:Time period,OBS_VALUE:Observation Value
1742,AFG: Afghanistan,MG_INTNL_MG_CNTRY_DEST: International migrants...,Y0T17: Under 18 years old,_T: Total,1990,15
1743,AFG: Afghanistan,MG_INTNL_MG_CNTRY_DEST: International migrants...,Y0T17: Under 18 years old,_T: Total,1995,18
1744,AFG: Afghanistan,MG_INTNL_MG_CNTRY_DEST: International migrants...,Y0T17: Under 18 years old,_T: Total,2000,19
1745,AFG: Afghanistan,MG_INTNL_MG_CNTRY_DEST: International migrants...,Y0T17: Under 18 years old,_T: Total,2005,22
1746,AFG: Afghanistan,MG_INTNL_MG_CNTRY_DEST: International migrants...,Y0T17: Under 18 years old,_T: Total,2010,25


In [15]:
# Utilizar la función pivot para convertir la columna "time_period" en columnas separadas
data_pivot = df.pivot(index=['REF_AREA:Geographic area', 'INDICATOR:Indicator', 'AGE:Current age', 'STAT_POP:Statistical Population'],
                        columns='TIME_PERIOD:Time period',
                        values='OBS_VALUE:Observation Value')

# Restablecer el índice del DataFrame resultante
df2 = data_pivot.reset_index()

# Imprimir el resultado
df2

TIME_PERIOD:Time period,REF_AREA:Geographic area,INDICATOR:Indicator,AGE:Current age,STAT_POP:Statistical Population,1990,1995,2000,2005,2008,2009,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,ABW: Aruba,MG_INTNL_MG_CNTRY_DEST: International migrants...,Y0T17: Under 18 years old,_T: Total,3,4,5,4,NaN,NaN,...,NaN,NaN,NaN,3,NaN,NaN,NaN,NaN,11,NaN
1,AFG: Afghanistan,MG_INTERNAL_DISP_PERS: Internally displaced pe...,Y0T17: Under 18 years old,POP_CONF_VIOLENCE: Share due to conflict and v...,NaN,NaN,NaN,NaN,0,150000,...,240000,310000,400000,580000,760000,630000,1300000,1500000,1700000,2100000
2,AFG: Afghanistan,MG_INTERNAL_DISP_PERS: Internally displaced pe...,Y0T17: Under 18 years old,POP_DISASTER: Share due to disaster,NaN,NaN,NaN,NaN,0,0,...,0,0,0,0,0,0,0,590000,550000,670000
3,AFG: Afghanistan,MG_INTERNAL_DISP_PERS: Internally displaced pe...,Y0T17: Under 18 years old,_T: Total,NaN,NaN,NaN,NaN,0,150000,...,240000,310000,400000,580000,760000,630000,1300000,2100000,2300000,2800000
4,AFG: Afghanistan,MG_INTNL_MG_CNTRY_DEST: International migrants...,Y0T17: Under 18 years old,_T: Total,15,18,19,22,NaN,NaN,...,NaN,NaN,NaN,139,NaN,NaN,NaN,NaN,54,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1094,ZWE: Zimbabwe,MG_INTERNAL_DISP_PERS: Internally displaced pe...,Y0T17: Under 18 years old,POP_CONF_VIOLENCE: Share due to conflict and v...,NaN,NaN,NaN,NaN,NaN,0,...,0,0,0,0,0,0,0,0,0,0
1095,ZWE: Zimbabwe,MG_INTERNAL_DISP_PERS: Internally displaced pe...,Y0T17: Under 18 years old,POP_DISASTER: Share due to disaster,NaN,NaN,NaN,NaN,NaN,0,...,0,0,0,0,0,0,0,25000,10000,21000
1096,ZWE: Zimbabwe,MG_INTERNAL_DISP_PERS: Internally displaced pe...,Y0T17: Under 18 years old,_T: Total,NaN,NaN,NaN,NaN,NaN,0,...,0,0,0,0,0,0,0,25000,10000,21000
1097,ZWE: Zimbabwe,MG_INTNL_MG_CNTRY_DEST: International migrants...,Y0T17: Under 18 years old,_T: Total,144,98,78,72,NaN,NaN,...,NaN,NaN,NaN,56,NaN,NaN,NaN,NaN,53,NaN


In [16]:
df2.columns

Index([       'REF_AREA:Geographic area',             'INDICATOR:Indicator',
                       'AGE:Current age', 'STAT_POP:Statistical Population',
                                    1990,                              1995,
                                    2000,                              2005,
                                    2008,                              2009,
                                    2010,                              2011,
                                    2012,                              2013,
                                    2014,                              2015,
                                    2016,                              2017,
                                    2018,                              2019,
                                    2020,                              2021],
      dtype='object', name='TIME_PERIOD:Time period')

In [17]:
df2.to_csv('predict_migration.csv', index=False)

In [46]:
# Cargar el DataFrame pivotado
df3 = pd.read_csv('predict_migration.csv')  # archivo CSV
df3


,REF_AREA:Geographic area,INDICATOR:Indicator,AGE:Current age,STAT_POP:Statistical Population,1990,1995,2000,2005,2008,2009,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,ABW: Aruba,MG_INTNL_MG_CNTRY_DEST: International migrants...,Y0T17: Under 18 years old,_T: Total,3,4,5,4,NaN,NaN,...,NaN,NaN,NaN,3,NaN,NaN,NaN,NaN,11,NaN
1,AFG: Afghanistan,MG_INTERNAL_DISP_PERS: Internally displaced pe...,Y0T17: Under 18 years old,POP_CONF_VIOLENCE: Share due to conflict and v...,NaN,NaN,NaN,NaN,0.0,150000.0,...,240000.0,310000.0,400000.0,580000,760000.0,630000.0,1300000.0,1500000.0,1700000,2100000.0
2,AFG: Afghanistan,MG_INTERNAL_DISP_PERS: Internally displaced pe...,Y0T17: Under 18 years old,POP_DISASTER: Share due to disaster,NaN,NaN,NaN,NaN,0.0,0.0,...,0.0,0.0,0.0,0,0.0,0.0,0.0,590000.0,550000,670000.0
3,AFG: Afghanistan,MG_INTERNAL_DISP_PERS: Internally displaced pe...,Y0T17: Under 18 years old,_T: Total,NaN,NaN,NaN,NaN,0.0,150000.0,...,240000.0,310000.0,400000.0,580000,760000.0,630000.0,1300000.0,2100000.0,2300000,2800000.0
4,AFG: Afghanistan,MG_INTNL_MG_CNTRY_DEST: International migrants...,Y0T17: Under 18 years old,_T: Total,15,18,19,22,NaN,NaN,...,NaN,NaN,NaN,139,NaN,NaN,NaN,NaN,54,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1094,ZWE: Zimbabwe,MG_INTERNAL_DISP_PERS: Internally displaced pe...,Y0T17: Under 18 years old,POP_CONF_VIOLENCE: Share due to conflict and v...,NaN,NaN,NaN,NaN,NaN,0.0,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0.0
1095,ZWE: Zimbabwe,MG_INTERNAL_DISP_PERS: Internally displaced pe...,Y0T17: Under 18 years old,POP_DISASTER: Share due to disaster,NaN,NaN,NaN,NaN,NaN,0.0,...,0.0,0.0,0.0,0,0.0,0.0,0.0,25000.0,10000,21000.0
1096,ZWE: Zimbabwe,MG_INTERNAL_DISP_PERS: Internally displaced pe...,Y0T17: Under 18 years old,_T: Total,NaN,NaN,NaN,NaN,NaN,0.0,...,0.0,0.0,0.0,0,0.0,0.0,0.0,25000.0,10000,21000.0
1097,ZWE: Zimbabwe,MG_INTNL_MG_CNTRY_DEST: International migrants...,Y0T17: Under 18 years old,_T: Total,144,98,78,72,NaN,NaN,...,NaN,NaN,NaN,56,NaN,NaN,NaN,NaN,53,NaN


In [48]:
# Paso 1: Limpiar los encabezados
df3.columns = df3.columns.str.split(':').str[-1].str.strip()

# Verificar las columnas presentes en el DataFrame
print(df3.columns)

# Paso 2: Asignar nuevos nombres a las columnas de años
columnas_anio = list(range(1990, 2022))
nuevas_columnas = ['REF_AREA', 'INDICATOR', 'AGE', 'STAT_POP'] + columnas_anio[:len(df3.columns) - 4]

# Verificar si las columnas en nuevas_columnas existen en el DataFrame
print(set(nuevas_columnas).issubset(df3.columns))


Index(['Geographic area', 'Indicator', 'Current age', 'Statistical Population',
       '1990', '1995', '2000', '2005', '2008', '2009', '2010', '2011', '2012',
       '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021'],
      dtype='object')
False
